### xarray wrapper for pyqg
https://github.com/pyqg/pyqg/issues/145

In [1]:
import numpy as np
import xarray as xr
import pyqg
from pyqg import diagnostic_tools

In [2]:
# help(pyqg.Model)
# help(pyqg.QGModel)
# help(pyqg.LayeredModel)
# help(pyqg.BTModel)
# help(pyqg.SQGModel)
# help(pyqg.LagrangianParticleArray2D)
# help(pyqg.GriddedLagrangianParticleArray2D)
# help(diagnostic_tools.calc_ispec)
# help(diagnostic_tools.spec_sum)
# help(diagnostic_tools.spec_var)

In [16]:
year = 24*60*60*360.
m = pyqg.QGModel(tmax=10*year, twrite=10000, tavestart=5*year)
m.run()

# for v in dir(m):
#     print(f"{v}:{type(getattr(m,v))}") 

INFO:  Logger initialized
INFO: Step: 10000, Time: 7.20e+07, KE: 4.71e-04, CFL: 0.095
INFO: Step: 20000, Time: 1.44e+08, KE: 5.16e-04, CFL: 0.092
INFO: Step: 30000, Time: 2.16e+08, KE: 5.00e-04, CFL: 0.104
INFO: Step: 40000, Time: 2.88e+08, KE: 4.65e-04, CFL: 0.089


In [47]:
def model_to_dataset(m):
    '''Convert outputs from model to an xarray dataset'''

   # Define dict for variable dimensions
    spatial_dims = ('time','z','y','x')
    spectral_dims = ('time','z','l','k')
    dim_database = {
        'q': spatial_dims,
        'u': spatial_dims,
        'v': spatial_dims,
        'ufull': spatial_dims,
        'vfull': spatial_dims, 
        'qh': spectral_dims,
        'uh': spectral_dims,
        'vh': spectral_dims,
        'ph': spectral_dims, 
        'Ubg': ('z'),
        'Qy': ('z'),
    }

    # dict for variable dimensions
    var_attr_database = {
        'q': {'long_name': 'potential vorticity in real space', 'units': 'm\u00B2 s\u207B\u00B9 K kg\u207B\u00B9',},
        'u': {'long_name': 'zonal velocity anomaly', 'units': 'm s\u207B\u00B9',},
        'v': {'long_name': 'meridional velocity anomaly', 'units': 'm s\u207B\u00B9',},
        'ufull': {'long_name': 'zonal full velocities in real space', 'units': 'm s\u207B\u00B9',},
        'vfull': {'long_name': 'meridional full velocities in real space', 'units': 'm s\u207B\u00B9',},
        'qh': {'long_name': 'potential vorticity in spectral space', 'units': 'm\u00B2 s\u207B\u00B9 K kg\u207B\u00B9',},
        'uh': {'long_name': 'zonal velocity anomaly in spectral space', 'units': 'm s\u207B\u00B9',},
        'vh': {'long_name': 'meridional velocity anomaly in spectral space', 'units': 'm s\u207B\u00B9',},
        'ph': {'long_name': 'streamfunction in spectral space', 'units': 'm\u00B2 s\u207B\u00B9',},
        'Ubg': {'long_name': 'background zonal velocity', 'units': 'm s\u207B\u00B9',},
        'Qy': {'long_name': 'background potential vorticity gradient', 'units': 'm\u00B2 s\u207B\u00B9 K kg\u207B\u00B9',} ,
        'kk': {'long_name': 'zonal wavenumbers', 'units': 'm\u207B\u00B9',} ,
        'll': {'long_name': 'meridional wavenumbers', 'units': 'm\u207B\u00B9',} ,
    }

    # dict for coordinate dimensions
    coord_database = {
        'time': ('time'),
        'z': ('z'),
        'x': ('x'),
        'y': ('y'),
        'l': ('l'),
        'k': ('k'),
        'nx': (),
        'ny': (),
        'nz': (),
        'nl': (),
        'nk': (),
        'rek': (),
        'tc': (),
        'dt': (),
        'L': (),
        'W': (),
        'filterfac': (),
        'twrite': (),
        'tmax': (),
        'tavestart': (),
        'tsnapstart': (),
        'taveint': (),
        'tsnapint': (),
        'ntd': (),
        'pmodes': (),
        'radii': (),
    }

    # dict for coordinate attributes 
    coord_attr_database = {
        'time': {'long_name': 'model time', 'units': 'seconds',},
        'z': {'long_name': 'vertical levels', 'units': 'none',},
        'x': {'long_name': 'real space grid points in the x direction', 'units': 'grid point',},
        'y': {'long_name': 'real space grid points in the y direction', 'units': 'grid point',},
        'l': {'long_name': 'spectal space grid points in the l direction', 'units': 'meridional wavenumber',},
        'k': {'long_name': 'spectal space grid points in the k direction', 'units': 'zonal wavenumber',},
        'nx': {'long_name': 'number of real space grid points in x direction', 'units': 'none',},
        'ny': {'long_name': 'number of real space grid points in y direction (default: nx)', 'units': 'none',},
        'nz': {'long_name': 'number of vertical levels', 'units': 'none',},
        'nl': {'long_name': 'number of spectral space grid points in l direction', 'units': 'grid point',},
        'nk': {'long_name': 'number of spectral space grid points in k direction', 'units': 'grid point',},
        'rek': {'long_name': 'linear drag in lower layer', 'units': 'seconds\u207B\u00B9',},
        'tc': {'long_name': 'model timestep', 'units': 'seconds',},
        'dt': {'long_name': 'numerical timestep', 'units': 'seconds',},
        'L': {'long_name': 'domain length in x direction', 'units': 'meters',},
        'W': {'long_name': 'domain length in y direction', 'units': 'meters',},
        'filterfac': {'long_name': 'amplitude of spectral spherical filter', 'units': '',},
        'twrite': {'long_name': 'interval for cfl writeout', 'units': 'number of timesteps',},
        'tmax': {'long_name': 'total time of integration', 'units': 'seconds',},
        'tavestart': {'long_name': 'start time for averaging', 'units': 'seconds',},
        'tsnapstart': {'long_name': 'start time for snapshot writeout', 'units': 'seconds'},
        'taveint': {'long_name': 'time interval for accumulation of diagnostic averages', 'units': 'seconds'},
        'tsnapint': {'long_name': 'time interval for snapshots', 'units': 'seconds',},
        'ntd': {'long_name': 'number of threads used', 'units': 'none',},
        'pmodes': {'long_name': 'vertical pressure modes', 'units': 'none',},
        'radii': {'long_name': 'deformation radii', 'units': 'meters',},
    }

    # dict for dataset attributes
    ds_attr_database = {
        'title': 'pyqg: Python Quasigeostrophic Model',
        'institution': '',
        'source': ('version: {}'.format(pyqg.__version__)),
        'history': '',
        'references': 'https://pyqg.readthedocs.io/en/latest/index.html',
        'comment': '', 
    }
    
    
    # Create list of variables
    variables = {}
    for vname in dim_database:
        data = getattr(m,vname)
        if 'time' in dim_database[vname]:
            variables[vname] = (dim_database[vname],data[np.newaxis,...])
        else:
            variables[vname] = (dim_database[vname],data)

    # Create list of coordinates
    coordinates = {}
    cname_to_1D = ['time','z','x','y','l','k']
    coords_1D = [(np.array([m.t])), (np.arange(1,m.nz+1)), (m.x[0,:]), (m.y[:,0]), (m.l[:,0]), (m.k[0,:])] 
    for cname in coord_database:
        if cname in cname_to_1D:
            index = cname_to_1D.index(cname)
            coordinates[cname] = coords_1D[index]
        else:
            try:
                data = getattr(m,cname)
                coordinates[cname] = (coord_database[cname],data )
            except:
                pass
        
    # Define dataset
    ds = xr.Dataset(variables,
                    coords=coordinates,
                    attrs=ds_attr_database)
    
    # Assign attributes to coordinates
    for caname in coord_attr_database:
        if caname in ds.coords:
            ds.coords[caname].attrs = coord_attr_database[caname]

    # Assign attributes to variables
    for vaname in var_attr_database:
        if vaname in ds.data_vars:
            ds.data_vars[vaname].attrs = var_attr_database[vaname]

            
    return ds
        

In [48]:
ds = model_to_dataset(m)
ds

<xarray.Dataset>
Dimensions:    (k: 33, l: 64, time: 1, x: 64, y: 64, z: 2)
Coordinates: (12/22)
  * time       (time) float64 3.11e+08
  * z          (z) int64 1 2
  * x          (x) float64 7.812e+03 2.344e+04 3.906e+04 ... 9.766e+05 9.922e+05
  * y          (y) float64 7.812e+03 2.344e+04 3.906e+04 ... 9.766e+05 9.922e+05
  * l          (l) float64 0.0 6.283e-06 1.257e-05 ... -1.257e-05 -6.283e-06
  * k          (k) float64 0.0 6.283e-06 1.257e-05 ... 0.0001948 0.0002011
    ...         ...
    filterfac  float64 23.6
    twrite     int64 10000
    tmax       float64 3.11e+08
    tavestart  float64 1.555e+08
    taveint    float64 8.64e+04
    ntd        int64 1
Data variables:
    q          (time, z, y, x) float64 3.307e-06 -4.046e-06 ... 8.907e-07
    u          (time, z, y, x) float64 0.07388 0.04794 ... 0.004435 0.00571
    v          (time, z, y, x) float64 0.03379 0.01486 ... 0.00121 0.003504
    ufull      (time, z, y, x) float64 0.09888 0.07294 ... 0.004435 0.00571
    vfull      (time, z, y, x) float64 0.03379 0.01486 ... 0.00121 0.003504
    qh         (time, z, l, k) complex128 (0.002069335318101613+0j) ... (8.40...
    uh         (time, z, l, k) complex128 (92.04672147654142+100.272944079510...
    vh         (time, z, l, k) complex128 (12.240144244324036-1.3798256257195...
    ph         (time, z, l, k) complex128 0j ... (-1.7670610282630347e-16+6.0...
    Ubg        (z) float64 0.025 0.0
    Qy         (z) float64 1.039e-10 -7.222e-12
Attributes:
    title:        pyqg: Python Quasigeostrophic Model
    institution:  
    source:       version: 0.3.0
    history:      
    references:   https://pyqg.readthedocs.io/en/latest/index.html
    comment: